<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_009_chatbots_yield_vs_return_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chatbots

### Install Libraries

In [ ]:
# !pip install python-dotenv
# !pip install openai
# !pip install google-generativeai
# !pip install anthropic
# !pip install gradio

### Import Libraries

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

### Load Environment Variables

In [ ]:
# Load the environment variables from the .env file
load_dotenv('/content/API_KEYS.env')  # Ensure this is the correct path to your file

# Get the API keys from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

# Check if the keys are loaded correctly and print a portion of them
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[0:10]}...")  # Only print part of the key
else:
    print("OpenAI API key not loaded correctly.")

if anthropic_api_key:
    print(f"Anthropic API Key loaded: {anthropic_api_key[0:10]}...")
else:
    print("Anthropic API key not loaded correctly.")

if google_api_key:
    print(f"Google API Key loaded: {google_api_key[0:10]}...")
else:
    print("Google API key not loaded correctly.")

OpenAI API Key loaded: sk-proj-e1...
Anthropic API Key loaded: sk-ant-api...
Google API Key loaded: AIzaSyDh3a...


In [ ]:
import openai
import anthropic
import google.generativeai

# Connect to OpenAI
openai.api_key = openai_api_key  # Set OpenAI API key

# Connect to Anthropic (Claude)
claude = anthropic.Anthropic(api_key=anthropic_api_key)  # Set Anthropic API key

# Connect to Google Generative AI
google.generativeai.configure(api_key=google_api_key)  # Set Google API key

#Customer Service Chatbot

Let's start building a **customer service chatbot** for Stanley's popular store, focusing on handling FAQ-style conversations. The chatbot will be able to handle common customer queries like product availability, returns, shipping, and possibly escalate issues to human support.

### Strategy:
- **Handle common questions**: FAQs about products, returns, shipping times, etc.
- **Escalation**: Detect when a user needs human support.
- **Sentiment analysis** (optional, advanced feature): Detect frustration and automatically offer human support or apologies.

### Step 1: Define Common Queries (FAQ)
We’ll build the chatbot to handle the following common queries:
- **Product Availability**: Is the Stanley Cup available in certain colors or sizes?
- **Returns**: How to return a product?
- **Shipping Times**: How long does shipping take?
- **Order Tracking**: Where is my order?
- **Escalation**: When the user asks for human support or gets frustrated.

### Step 2: Implement the Chatbot Functionality
We’ll create a basic chatbot function that can answer these queries based on the user's input. Later, we can enhance it with sentiment analysis and multilingual support.

### How It Works:
1. **Predefined FAQs**: The chatbot has built-in answers for common customer service queries like product availability, returns, shipping, and order tracking. If the user's message contains keywords related to those topics, it will return a pre-programmed response.
   
2. **Handling Complex Queries**: If the user asks a question that doesn't match the predefined FAQs, the chatbot passes the query to OpenAI's GPT model to generate a response.

3. **Escalation to Human Support**: If the user mentions wanting to speak with a human representative, the chatbot will offer an escalation message, informing the user that a human can be connected.

### Next Steps:
1. **Sentiment Analysis**: We can integrate a basic sentiment analysis function to detect frustration (e.g., if the user is unhappy with the bot's answers) and automatically suggest human assistance.
   
2. **Multilingual Support**: Later, we can add language detection and handle conversations in multiple languages.

3. **API Integration**: For order tracking or real-time availability, we can integrate actual APIs from Stanley’s website or database to provide more personalized responses.



## Return Response

In [ ]:
import gradio as gr

MODEL = 'gpt-4o-mini'
system_message = "You are a helpful customer service assistant for the Stanley store. You assist customers with queries about products, returns, shipping, and orders."
history_file = "chat_history.json"

# Function to save chat history to a JSON file
def save_chat_history(history, file_path):
    with open(file_path, "w") as file:
        json.dump(history, file, indent=4)

# Sample FAQs for the Stanley store
faq_responses = {
    "availability": "The Stanley Cup is available in multiple colors and sizes, but some items may be out of stock due to high demand. You can check the latest availability on our website.",
    "returns": "You can return any Stanley product within 30 days of purchase. Just visit our Returns page for more details and to start the process.",
    "shipping": "Shipping typically takes between 3-5 business days for standard delivery. Expedited options are also available.",
    "order tracking": "You can track your order by visiting the 'Order Tracking' page on our website. Just enter your order number to get the latest updates.",
    "human support": "I can help you with most questions, but if you need further assistance, I can connect you to a human representative."
}

# Chat function
def stanley_chat(message, history):
    # System message at the start
    messages = [{"role": "system", "content": system_message}]

    # Add the conversation history
    for msg in history:
        messages.append(msg)

    # Add the current user message
    messages.append({"role": "user", "content": message})

    # Check for common FAQs
    response = ""
    if "availability" in message.lower():
        response = faq_responses["availability"]
    elif "return" in message.lower():
        response = faq_responses["returns"]
    elif "shipping" in message.lower():
        response = faq_responses["shipping"]
    elif "track" in message.lower():
        response = faq_responses["order tracking"]
    elif "human" in message.lower() or "representative" in message.lower():
        response = faq_responses["human support"]
    else:
        # Use the AI to handle more complex or unknown questions
        stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''

    return response

    # Add the current user message and assistant response to the history
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})

    # Save updated history to a JSON file
    save_chat_history(history, history_file)

# Gradio interface
gr.ChatInterface(fn=stanley_chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://668330e9b473570394.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Read Chat Log

In [ ]:
import json
import os

def print_chat_history(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            history = json.load(file)

        print("\nChat History:\n" + "=" * 50)
        turn = 1
        for msg in history:
            if msg["role"] == "user":
                # print(f"Turn {turn}:")
                print(f"User: {msg['content']}")
            elif msg["role"] == "assistant":
                print(f"Assistant: {msg['content']}")
                print("-" * 50)
                turn += 1
    else:
        print("No chat history found.")

print_chat_history("/content/chat_history.json")


Chat History:
User: hello
Assistant: Hello! How can I assist you today?
--------------------------------------------------
User: i need some help with a return
Assistant: You can return any Stanley product within 30 days of purchase. Just visit our Returns page for more details and to start the process.
--------------------------------------------------


## Yield (Stream) Response

You can modify the function to consistently use `yield` even when an FAQ response is returned. Now, if the FAQ condition is met, the response is yielded directly, maintaining a consistent generator flow in both cases.

```python
  # If the question is not in FAQs, generate response from the AI
  if not response:
      stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
      for chunk in stream:
          response += chunk.choices[0].delta.content or ''
          yield response
  else:
      # Yield the FAQ response as a single output
      yield response
```


In [ ]:
import gradio as gr
import json
import openai

MODEL = 'gpt-4o-mini'
system_message = "You are a helpful customer service assistant for the Stanley store. You assist customers with queries about products, returns, shipping, and orders."
history_file = "chat_history.json"

# Function to save chat history to a JSON file
def save_chat_history(history, file_path):
    with open(file_path, "w") as file:
        json.dump(history, file, indent=4)

# Sample FAQs for the Stanley store
faq_responses = {
    "availability": "The Stanley Cup is available in multiple colors and sizes, but some items may be out of stock due to high demand. You can check the latest availability on our website.",
    "returns": "You can return any Stanley product within 30 days of purchase. Just visit our Returns page for more details and to start the process.",
    "shipping": "Shipping typically takes between 3-5 business days for standard delivery. Expedited options are also available.",
    "order tracking": "You can track your order by visiting the 'Order Tracking' page on our website. Just enter your order number to get the latest updates.",
    "human support": "I can help you with most questions, but if you need further assistance, I can connect you to a human representative."
}

# Chat function
def chat(message, history):
    # Start the message list with the system message
    messages = [{"role": "system", "content": system_message}]

    # Iterate through the history, adding user and assistant messages using OpenAI-style format
    for msg in history:
        messages.append(msg)

    # Add the latest user message to the conversation
    messages.append({"role": "user", "content": message})

    # Check for common FAQs first
    response = ""
    if "availability" in message.lower():
        response = faq_responses["availability"]
    elif "return" in message.lower():
        response = faq_responses["returns"]
    elif "shipping" in message.lower():
        response = faq_responses["shipping"]
    elif "track" in message.lower():
        response = faq_responses["order tracking"]
    elif "human" in message.lower() or "representative" in message.lower():
        response = faq_responses["human support"]

    # If the question is not in FAQs, generate response from the AI
    if not response:
        stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            yield response
    else:
        # Yield the FAQ response as a single output
        yield response


    # Add the current user message and assistant response to the history
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})

    # Save updated history to a JSON file
    save_chat_history(history, history_file)

# Gradio interface
gr.ChatInterface(fn=chat, type="messages").launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bdda640b6f5d6228cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7866 <> https://bdda640b6f5d6228cf.gradio.live


### Read Chat Log

In [ ]:
import json
import os

def print_chat_history(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            history = json.load(file)

        print("\nChat History:\n" + "=" * 50)
        turn = 1
        for msg in history:
            if msg["role"] == "user":
                # print(f"Turn {turn}:")
                print(f"User: {msg['content']}")
            elif msg["role"] == "assistant":
                print(f"Assistant: {msg['content']}")
                print("-" * 50)
                turn += 1
    else:
        print("No chat history found.")

print_chat_history("/content/chat_history.json")


Chat History:
User: hello
Assistant: Hello! How can I assist you today?
--------------------------------------------------
User: I need to make a return
Assistant: You can return any Stanley product within 30 days of purchase. Just visit our Returns page for more details and to start the process.
--------------------------------------------------
